<font color='green'> 🎄🎄🎄 Bonus: Get/stay festive 🎄🎄🎄</font><br> 
===========
Devices like the Pico have built in electronics for generating block waves. They use this generator for [Pulse Width Modulation](https://en.wikipedia.org/wiki/Pulse-width_modulation) (PWM). In PWM, the output signal of a digital pin oscillates on (+3.3 V) to off (0V) rapidly. The frequency and the duty cycle can be changed to modulate the signal. PWM is used in many applications, like dimming LEDs or controlling servo motors. You also can use PWM to send a pulsed signal to the buzzer and make it create a melody!

## Build the circuit:
- Connect `Dout0 - GP14` to the `BUZZER` pin.

## Run the code:
Below, you'll find an adaptation of code which can be used to play tunes on the ALPACA. The original code was made by `james1236` and can be found on [GitHub](https://github.com/james1236/buzzer_music). There are three cells: 
1. The first cell defines the functions that play the music. Run this once.
2. The second cell stores your music "file". Run this once.
3. The third cell will play the tune.

### Setup Code

In [ ]:
%serialconnect to --port="COM8" 

In [ ]:
from machine import Pin, PWM
from math import ceil
from time import sleep

tones = { 'C0':16, 'C#0':17, 'D0':18, 'D#0':19, 'E0':21, 'F0':22, 'F#0':23, 'G0':24, 'G#0':26, 'A0':28, 'A#0':29, 'B0':31, 'C1':33, 'C#1':35, 'D1':37, 'D#1':39, 'E1':41, 'F1':44, 'F#1':46, 'G1':49, 'G#1':52, 'A1':55, 'A#1':58, 'B1':62, 'C2':65, 'C#2':69, 'D2':73, 'D#2':78, 'E2':82, 'F2':87, 'F#2':92, 'G2':98, 'G#2':104, 'A2':110, 'A#2':117, 'B2':123, 'C3':131, 'C#3':139, 'D3':147, 'D#3':156, 'E3':165, 'F3':175, 'F#3':185, 'G3':196, 'G#3':208, 'A3':220, 'A#3':233, 'B3':247, 'C4':262, 'C#4':277, 'D4':294, 'D#4':311, 'E4':330, 'F4':349, 'F#4':370, 'G4':392, 'G#4':415, 'A4':440, 'A#4':466, 'B4':494, 'C5':523, 'C#5':554, 'D5':587, 'D#5':622, 'E5':659, 'F5':698, 'F#5':740, 'G5':784, 'G#5':831, 'A5':880, 'A#5':932, 'B5':988, 'C6':1047, 'C#6':1109, 'D6':1175, 'D#6':1245, 'E6':1319, 'F6':1397, 'F#6':1480, 'G6':1568, 'G#6':1661, 'A6':1760, 'A#6':1865, 'B6':1976, 'C7':2093, 'C#7':2217, 'D7':2349, 'D#7':2489, 'E7':2637, 'F7':2794, 'F#7':2960, 'G7':3136, 'G#7':3322, 'A7':3520, 'A#7':3729, 'B7':3951, 'C8':4186, 'C#8':4435, 'D8':4699, 'D#8':4978, 'E8':5274, 'F8':5588, 'F#8':5920, 'G8':6272, 'G#8':6645, 'A8':7040, 'A#8':7459, 'B8':7902, 'C9':8372, 'C#9':8870, 'D9':9397, 'D#9':9956, 'E9':10548, 'F9':11175, 'F#9':11840, 'G9':12544, 'G#9':13290, 'A9':14080, 'A#9':14917, 'B9':15804 } 

#Time, Note, Duration, Instrument (onlinesequencer.net schematic format)
#0 D4 8 0;0 D5 8 0;0 G4 8 0;8 C5 2 0;10 B4 2 0;12 G4 2 0;14 F4 1 0;15 G4 17 0;16 D4 8 0;24 C4 8 0

class music:
    def __init__(self, songString='0 D4 8 0', looping=True, tempo=3, duty=2512, pin=None, pins=[Pin(0)]):
        self.tempo = tempo
        self.song = songString
        self.looping = looping
        self.duty = duty
        self.stopped = False
        self.timer = -1
        self.beat = -1
        self.arpnote = 0
        self.pwms = []
        
        if (not (pin is None)): pins = [pin]
            
        i = 0
        for pin in pins:
            self.pwms.append(PWM(pins[i]))
            i = i + 1
        
        self.notes = []

        self.playingNotes = []
        self.playingDurations = []

        #Find the end of the song
        self.end = 0
        splitSong = self.song.split(";")
        for note in splitSong:
            snote = note.split(" ")
            testEnd = round(float(snote[0])) + ceil(float(snote[2]))
            if (testEnd > self.end): self.end = testEnd
                
        #Create empty song structure
        while (self.end > len(self.notes)): self.notes.append(None)

        #Populate song structure with the notes
        for note in splitSong:
            snote = note.split(" ")
            beat = round(float(snote[0]));
            
            if (self.notes[beat] == None): self.notes[beat] = []
            self.notes[beat].append([snote[1],ceil(float(snote[2]))]) #Note, Duration

        #Round up end of song to nearest bar
        self.end = ceil(self.end / 8) * 8
    
    def stop(self):
        for pwm in self.pwms: pwm.deinit()
        self.stopped = True
        
    def tick(self):
        if (not self.stopped): 
            self.timer = self.timer + 1

            #Loop
            if (self.timer % (self.tempo * self.end) == 0 and (not (self.timer == 0))):
                if (not self.looping):
                    self.stop()
                    return False
                self.beat, self.timer = (-1, 0)
            
            #On Beat
            if (self.timer % self.tempo == 0): 
                self.beat = self.beat + 1

                #Remove expired notes from playing list
                i = 0
                while (i < len(self.playingDurations)):
                    self.playingDurations[i] = self.playingDurations[i] - 1
                    if (self.playingDurations[i] <= 0):
                        self.playingNotes.pop(i)
                        self.playingDurations.pop(i)
                    else: i = i + 1
                        
                #Add new notes and their durations to the playing list
                if (self.beat < len(self.notes)):
                    if (self.notes[self.beat] != None):
                        for note in self.notes[self.beat]:
                            self.playingNotes.append(note[0])
                            self.playingDurations.append(note[1])
                
                #Only need to run these checks on beats
                i = 0
                for pwm in self.pwms:
                    if (i >= len(self.playingNotes)): pwm.duty_u16(0)
                    else:
                        #Play note
                        pwm.duty_u16(self.duty)
                        pwm.freq(tones[self.playingNotes[i]])
                    i = i + 1
            
            #Play arp of all playing notes
            if (len(self.playingNotes) > len(self.pwms)):
                self.pwms[len(self.pwms)-1].duty_u16(self.duty)
                if (self.arpnote > len(self.playingNotes)-len(self.pwms)):
                    self.arpnote = 0
                self.pwms[len(self.pwms)-1].freq(tones[self.playingNotes[self.arpnote+(len(self.pwms)-1)]])
                self.arpnote = self.arpnote + 1
                
            return True
        else:
            return False


### Set song

In [ ]:
songA = '0 A5 6 41;8 A5 2 41;10 G5 4 41;14 D5 2 41;6 A5 2 41;16 A5 2 41;18 A5 2 41;20 B5 2 41;22 G5 2 41;24 G5 4 41;28 E5 2 41;30 E5 2 41;32 A5 2 41;34 A5 2 41;36 B5 4 41;40 G5 4 41;46 G5 2 41;50 G5 2 41;54 E5 2 41;56 E5 4 41;48 F#5 2 41;52 F#5 2 41;64 B5 6 41;72 A5 6 41;78 E5 2 41;70 A5 2 41;80 B5 2 41;82 C6 2 41;84 B5 2 41;86 A5 2 41;88 A5 4 41;94 E5 2 41;96 F#5 2 41;98 G5 2 41;100 F#5 2 41;102 F#5 2 41;104 F#5 2 41;106 G5 4 41;110 F#5 2 41;112 F#5 2 41;114 D5 6 41'
songB = '0 D5 4 41;4 G5 2 41;7 G5 1 41;8 G5 4 41;12 A5 4 41;16 B5 2 41;19 B5 1 41;20 B5 6 41;26 B5 2 41;28 A5 2 41;30 B5 2 41;32 C6 4 41;36 F#5 4 41;40 A5 4 41;44 G5 8 41;96 G5 8 41;52 D5 4 41;56 G5 2 41;59 G5 1 41;60 G5 4 41;64 A5 4 41;68 B5 2 41;71 B5 1 41;72 B5 6 41;78 B5 2 41;80 A5 2 41;82 B5 2 41;84 C6 4 41;88 F#5 4 41;92 A5 4 41;104 D6 1 41;106 D6 2 41;108 C6 2 41;110 E6 6 41;116 D6 1 41;118 D6 2 41;120 C6 1 41;122 C6 6 41;128 C6 1 41;130 C6 2 41;132 A5 2 41;134 D6 6 41;140 C6 1 41;142 C6 2 41;144 B5 1 41;146 B5 6 41;196 G5 8 41;152 D5 4 41;156 G5 2 41;159 G5 1 41;160 G5 4 41;164 A5 4 41;168 B5 2 41;171 B5 1 41;172 B5 6 41;178 B5 2 41;180 A5 2 41;182 B5 2 41;184 C6 4 41;188 F#5 4 41;192 A5 4 41;4 G3 8 41;12 D3 4 41;16 G3 12 41;28 D3 8 41;36 D3 8 41;44 G3 8 41'
songC = '2 A5 2 0;4 B5 2 0;6 A5 2 0;10 G#5 2 0;12 B5 2 0;14 G#5 2 0;18 F#5 2 0;20 B5 2 0;22 F#5 2 0;26 G#5 2 0;28 B5 2 0;30 G#5 2 0;34 A5 2 0;36 B5 2 0;38 A5 2 0;42 G#5 2 0;44 B5 2 0;46 G#5 2 0;50 A5 2 0;52 B5 2 0;54 A5 2 0;58 G#5 2 0;60 B5 2 0;62 G#5 2 0;68 C#5 1 0;69.33000183105469 D5 1 0;70.66999816894531 C#5 1 0;72 B4 3 0;74.66999816894531 A4 3 0;78 A4 1 0;79 B4 1 0;80 C#5 2 0;82 E5 2 0;84 A5 2 0;86 C#6 2 0;88 G#5 4 0;92 G#5 3 0;95 A5 1 0;96 F#5 6 0;102 F#5 1 0;103 G#5 1 0;104 E5 6 0;110 A5 1 0;111 G#5 1 0;112 G#5 3 0;115 F#5 1 0;116 F#5 2 0;118 E5 2 0;120 A4 2 0;120 B4 2 0;120 E5 2 0;122 G#4 2 0;124 A4 3 0;127 B4 1 0;128 C#5 6 0;134 D5 1 0;135 C#5 1 0;136 B4 2 0;138 A4 4 0;142 A4 1 0;143 B4 1 0;144 C#5 2 0;146 E5 2 0;148 A5 2 0;150 C#6 2 0;152 G#5 4 0;156 G#5 3 0;159 A5 1 0;160 F#5 6 0;166 F#5 1 0;167 G#5 1 0;168 E5 6 0;174 A5 1 0;175 G#5 1 0;176 G#5 3 0;179 F#5 1 0;180 F#5 2 0;182 E5 2 0;184 E5 11 0;196 C#5 1 0;197 D5 1 0;198 E5 2 0;200 A5 4 0;200 C#6 4 0;204 C#5 1 0;205 D5 1 0;206 E5 2 0;208 C#5 4 0;208 F5 4 0;208 G#5 4 0;208 B5 4 0;212 C#5 1 0;213 D#5 1 0;214 F5 2 0;218 C#5 4 0;218 F#5 4 0;218 A5 4 0;222 G#5 1 0;223 D5 3 0;223 F#5 3 0;226 G#5 2 0;228 F#5 2 0;230 E5 1 0;231 E5 5 0;232 F#4 4 0;232 A#4 4 0;232 C#5 4 0;236 A#4 2 0;236 C#5 2 0;236 F#5 2 0;238 E5 1 0;239 E5 3 0;242 F#5 2 0;244 E5 2 0;246 D5 1 0;247 A#4 3 0;247 C#5 3 0;250 A#4 2 0;250 C#5 2 0;252 D5 2 0;254 E5 2 0;256 A4 1 0;256 B4 1 0;256 E5 1 0;257 D5 1 0;258 C#5 1 0;259 B4 5 0;268 C#5 1 0;269 D5 1 0;270 E5 2 0;272 A5 4 0;272 C#6 4 0;276 C#5 1 0;277 D5 1 0;278 E5 2 0;280 C#5 4 0;280 F5 4 0;280 G#5 4 0;280 B5 4 0;284 C#5 1 0;285 D#5 1 0;286 F5 2 0;290 C#5 4 0;290 F#5 4 0;290 A5 4 0;294 G#5 1 0;295 D5 3 0;295 F#5 3 0;298 G#5 2 0;300 F#5 2 0;302 C#5 1 0;302 E5 1 0;303 C#5 5 0;303 E5 5 0;308 A#4 2 0;308 C#5 2 0;308 E5 2 0;310 E5 1 0;311 E5 3 0;314 F#5 2 0;316 E5 2 0;318 A#4 4 0;318 D5 4 0;322 C#5 2 0;324 D5 2 0;326 E5 2 0;328 A4 1 0;328 B4 1 0;328 E5 1 0;329 D5 1 0;330 C#5 1 0;331 B4 5 0;340 G#4 2 0;340 E5 2 0;342 E5 2 0;344 A4 1 0;344 B4 1 0;344 E5 1 0;345 D5 1 0;346 C#5 1 0;347 B4 5 0;364 C#5 1 0;365.3299865722656 D5 1 0;366.6700134277344 C#5 1 0;368 B4 3 0;370.6700134277344 A4 3 0;374 A4 1 0;375 B4 1 0;376 C#5 2 0;378 E5 2 0;380 A5 2 0;382 C#6 2 0;384 G#5 4 0;388 G#5 3 0;391 A5 1 0;392 F#5 6 0;398 F#5 1 0;399 G#5 1 0;400 E5 6 0;406 A5 1 0;407 G#5 1 0;408 G#5 3 0;411 F#5 1 0;412 F#5 2 0;414 E5 2 0;416 A4 2 0;416 B4 2 0;416 E5 2 0;418 G#4 2 0;420 A4 3 0;423 B4 1 0;424 C#5 6 0;430 D5 1 0;431 C#5 1 0;432 B4 2 0;434 A4 4 0;438 A4 1 0;439 B4 1 0;440 C#5 2 0;442 E5 2 0;444 A5 2 0;446 C#6 2 0;448 G#5 4 0;452 G#5 3 0;455 A5 1 0;456 F#5 6 0;462 F#5 1 0;463 G#5 1 0;464 E5 6 0;470 A5 1 0;471 G#5 1 0;472 G#5 3 0;475 F#5 1 0;476 F#5 2 0;478 E5 2 0;480 E5 11 0;492 C#5 1 0;493 D5 1 0;494 E5 2 0;496 A5 4 0;496 C#6 4 0;500 C#5 1 0;501 D5 1 0;502 E5 2 0;504 C#5 4 0;504 F5 4 0;504 G#5 4 0;504 B5 4 0;508 C#5 1 0;509 D#5 1 0;510 F5 2 0;514 C#5 4 0;514 F#5 4 0;514 A5 4 0;518 G#5 1 0;519 D5 3 0;519 F#5 3 0;522 G#5 2 0;524 F#5 2 0;526 E5 1 0;527 E5 5 0;528 F#4 4 0;528 A#4 4 0;528 C#5 4 0;532 A#4 2 0;532 C#5 2 0;532 F#5 2 0;534 E5 1 0;535 E5 3 0;538 F#5 2 0;540 E5 2 0;542 D5 1 0;543 A#4 3 0;543 C#5 3 0;546 A#4 2 0;546 C#5 2 0;548 D5 2 0;550 E5 2 0;552 A4 1 0;552 B4 1 0;552 E5 1 0;553 D5 1 0;554 C#5 1 0;555 B4 5 0;564 C#5 1 0;565 D5 1 0;566 E5 2 0;568 A5 4 0;568 C#6 4 0;572 C#5 1 0;573 D5 1 0;574 E5 2 0;576 C#5 4 0;576 F5 4 0;576 G#5 4 0;576 B5 4 0;580 C#5 1 0;581 D#5 1 0;582 F5 2 0;586 C#5 4 0;586 F#5 4 0;586 A5 4 0;590 G#5 1 0;591 D5 3 0;591 F#5 3 0;594 G#5 2 0;596 F#5 2 0;598 C#5 1 0;598 E5 1 0;599 C#5 5 0;599 E5 5 0;604 A#4 2 0;604 C#5 2 0;604 E5 2 0;606 E5 1 0;607 E5 3 0;610 F#5 2 0;612 E5 2 0;614 A#4 4 0;614 D5 4 0;618 C#5 2 0;620 D5 2 0;622 E5 2 0;624 A4 1 0;624 B4 1 0;624 E5 1 0;625 D5 1 0;626 C#5 1 0;627 B4 5 0;636 G#4 2 0;636 E5 2 0;638 E5 2 0;640 A4 1 0;640 B4 1 0;640 E5 1 0;641 D5 1 0;642 C#5 1 0;643 B4 12 0;656 A4 1 0;656 B4 1 0;656 E5 1 0;657 D5 1 0;658 C#5 1 0;659 B4 12 0;672 A4 1 0;672 D5 1 0;673 C#5 1 0;674 B4 1 0;675 A4 28 0;704 A4 8 0;704 E5 8 0;704 A5 8 0;0 A3 7 0;7 A3 1 0;8 A4 8 0;16 A4 8 0;24 A3 5 0;29 E4 2 0;31 E4 1 0;32 A3 8 0;40 A4 5 0;45 E4 2 0;47 E4 1 0;48 A3 7 0;55 A3 1 0;56 A4 8 0;64 A3 4 0;68 E4 4 0;68 A4 4 0;72 B3 4 0;76 F4 4 0;76 A4 4 0;80 C#4 4 0;84 E4 4 0;84 A4 4 0;88 C#4 4 0;88 E4 4 0;88 G#4 4 0;88 B4 4 0;92 C#4 4 0;96 D4 4 0;100 A4 4 0;100 D5 4 0;104 C#4 4 0;108 E4 4 0;108 A4 4 0;112 B3 4 0;112 F#4 4 0;112 B4 4 0;116 B3 4 0;116 D4 4 0;116 A4 4 0;120 E4 4 0;124 E3 4 0;128 A2 4 0;132 E4 4 0;132 A4 4 0;136 B2 4 0;140 F4 4 0;140 A4 4 0;144 C#3 4 0;148 E4 4 0;148 A4 4 0;152 C#3 4 0;156 E4 4 0;156 G#4 4 0;160 D3 4 0;164 F#4 4 0;164 A4 4 0;168 C#3 4 0;172 E4 4 0;172 A4 4 0;176 B2 4 0;180 D4 4 0;180 F#4 4 0;180 A4 4 0;184 E3 4 0;188 F#4 4 0;188 A4 4 0;188 B4 4 0;188 D5 4 0;192 E4 2 0;192 A4 2 0;192 D5 2 0;194 G#4 2 0;196 E3 4 0;200 A2 4 0;204 A4 4 0;208 F3 4 0;212 F4 4 0;216 F#3 4 0;220 F#4 4 0;224 D3 4 0;228 D4 4 0;232 A#3 4 0;236 A#3 4 0;240 A#2 4 0;244 A#3 4 0;248 F#3 4 0;252 A#2 4 0;256 B2 4 0;260 B3 4 0;264 E3 4 0;268 E3 4 0;272 A2 4 0;276 A4 4 0;280 F3 4 0;284 F4 4 0;288 F#3 4 0;292 F#4 4 0;296 D3 4 0;300 D4 4 0;304 A#2 4 0;308 A#3 4 0;312 A#2 4 0;316 A#3 4 0;320 F#3 4 0;324 A#3 4 0;328 B2 4 0;332 B3 4 0;336 E3 3 0;339 E3 1 0;340 E3 4 0;344 B3 4 0;348 B2 4 0;352 E3 3 0;355 E3 1 0;356 E4 4 0;360 A3 4 0;364 E4 4 0;364 A4 4 0;368 B3 4 0;372 F4 4 0;372 A4 4 0;376 C#4 4 0;380 E4 4 0;380 A4 4 0;384 C#4 4 0;384 E4 4 0;384 G#4 4 0;384 B4 4 0;388 C#4 4 0;392 D4 4 0;396 A4 4 0;396 D5 4 0;400 C#4 4 0;404 E4 4 0;404 A4 4 0;408 B3 4 0;408 F#4 4 0;408 B4 4 0;412 B3 4 0;412 D4 4 0;412 A4 4 0;416 E4 4 0;420 E3 4 0;424 A2 4 0;428 E4 4 0;428 A4 4 0;432 B2 4 0;436 F4 4 0;436 A4 4 0;440 C#3 4 0;444 E4 4 0;444 A4 4 0;448 C#3 4 0;452 E4 4 0;452 G#4 4 0;456 D3 4 0;460 F#4 4 0;460 A4 4 0;464 C#3 4 0;468 E4 4 0;468 A4 4 0;472 B2 4 0;476 D4 4 0;476 F#4 4 0;476 A4 4 0;480 E3 4 0;484 F#4 4 0;484 A4 4 0;484 B4 4 0;484 D5 4 0;488 E4 2 0;488 A4 2 0;488 D5 2 0;490 G#4 2 0;492 E3 4 0;496 A2 4 0;500 A4 4 0;504 F3 4 0;508 F4 4 0;512 F#3 4 0;516 F#4 4 0;520 D3 4 0;524 D4 4 0;528 A#3 4 0;532 A#3 4 0;536 A#2 4 0;540 A#3 4 0;544 F#3 4 0;548 A#2 4 0;552 B2 4 0;556 B3 4 0;560 E3 4 0;564 E3 4 0;568 A2 4 0;572 A4 4 0;576 F3 4 0;580 F4 4 0;584 F#3 4 0;588 F#4 4 0;592 D3 4 0;596 D4 4 0;600 A#2 4 0;604 A#3 4 0;608 A#2 4 0;612 A#3 4 0;616 F#3 4 0;620 A#3 4 0;624 B2 4 0;628 B3 4 0;632 E3 3 0;635 E3 1 0;636 E3 4 0;640 B3 4 0;644 B3 3 0;647 B3 1 0;648 E3 3 0;651 E3 1 0;652 E4 1 0;653 E4 3 0;656 B2 4 0;660 B3 1 0;661 B3 2 0;663 B3 1 0;664 E3 3 0;667 E3 1 0;668 C#4 1 0;669 C#4 3 0;672 A3 4 0;676 A2 4 0;680 A3 3 0;683 B3 1 0;684 C#4 4 0;688 C#4 1 0;689 C#4 2 0;691 D4 1 0;692 E4 4 0;696 A2 6 0;696 A3 6 0;702 A2 17 0;702 A3 17 0;72 D#7 1 43'

song=songC # or B or C, have fun

### Play

In [ ]:
mySong = music(song, pins=[Pin(14)])

while True:
    mySong.tick()
    sleep(0.04)

# Use the Interrupt (Stop) button to exit the while loop. 


In [ ]:
# The buzzer will stay on as the PWM pin is still turned on. Silence it using :
pwm = PWM(Pin(14))
pwm.duty_u16(0)

## Play/create your own music
Any song can be played this way. Try adding your own:
* Find some music on [onlinesequencer.net](https://onlinesequencer.net/)
* Click edit, select all notes with `CTRL + A` and then copy them with `CTRL + C` 
* Come back to this Jupyter Notebook and paste the string in place of the one in the *Set Song* cell. Alternatively, create a second cell and run it. See below.
* Make sure to remove the "Online Sequencer:120233:" from the start and the ";:" from the end

#### Example: (try to guess the song)

In [ ]:
songD = '0 G5 4 41;8 D6 4 41;16 G6 4 41;20 F#6 4 41;32 G5 4 41;36 C6 4 41;44 G6 4 41;0 G4 16 41;16 E4 16 41;32 C4 16 41;48 D5 16 41;4 B5 4 41;12 F#6 4 41;60 A5 4 41;64 G5 4 41;68 B5 4 41;64 G4 16 41;48 D6 12 41;78 F#6 2 41;80 G6 4 41;84 F#6 4 41;88 E6 4 41;92 D6 4 41;80 G4 16 41;96 G4 16 41;104 F#6 4 41;108 G6 2 41;112 F#6 4 41;116 G6 2 41;120 E6 4 41;112 G4 16 41;124 D6 4 41;96 A6 1 41;128 C4 32 41;134 E6 2 41;136 G6 4 41;140 A6 2 41;142 B6 6 41;132 C6 2 41;156 E6 2 41;148 A6 4 41;152 G6 4 41;160 B6 2 41;162 D7 2 41;164 B6 2 41;160 C4 32 41;168 G6 4 41;172 A6 4 41;176 G6 4 41;188 D6 4 41;180 A6 4 41;192 G6 4 41;196 A6 4 41;200 F#6 4 41;204 G6 4 41;212 F#6 4 41;184 D#6 4 41;192 G4 16 41;208 B4 16 41;224 E4 16 41;240 E4 8 41;248 C4 8 41;208 E6 4 41;224 B6 4 41;230 A6 6 41;228 A6 2 41;236 G6 4 41;240 F#6 4 41;244 G6 4 41;256 G4 16 41;72 D6 4 41;100 G6 2 41;260 D6 2 41;262 E6 2 41;264 G6 4 41;268 D7 4 41;272 G#4 16 41;288 A4 16 41;288 B6 4 41;292 A6 4 41;296 G6 4 41;300 E6 4 41;304 D#6 4 41;304 C5 16 41;316 B6 4 41;308 A6 8 41;320 G4 16 41;320 G6 16 41;336 G6 4 41;340 G6 4 41;336 D6 4 41;340 D6 4 41;344 D6 4 41;348 D6 4 41;352 E6 4 41;356 E6 4 41;360 E6 4 41;364 E6 4 41;344 G6 4 41;348 G6 4 41;352 G6 4 41;356 G6 4 41;360 G6 4 41;364 G6 4 41;368 C6 4 41;368 G6 4 41;372 G6 4 41;376 G6 4 41;372 C6 4 41;376 C6 4 41;380 C6 4 41;387 D6 1 41;391 D6 1 41;395 D6 1 41;399 D6 1 41;388 D6 3 41;392 D6 3 41;396 D6 3 41;380 G6 4 41;24 E6 2 41;26 D6 2 41;28 B5 2 41;30 A5 2 41;40 E6 4 41;216 D#6 4 41;220 D6 4 41;248 D#6 4 41;252 D6 4 41;272 C7 12 41;286 C7 2 41;284 B6 2 41;400 G4 16 41;400 G5 4 41;412 F#6 2 41;404 B5 4 41;408 D6 2 41;414 G6 4 41;416 G4 16 41;418 F#6 4 41;422 E6 4 41;426 D6 4 41;432 A6 4 41;432 G4 16 41;436 G6 4 41;440 G6 2 41;444 F#6 2 41;446 G6 4 41;450 F#6 4 41;454 D6 6 41;448 G4 16 41;460 D6 2 41;462 C6 3 41;468 E6 4 41;472 G6 2 41;464 C5 16 41;476 A6 2 41;478 B6 6 41;484 A6 4 41;488 G6 4 41;492 E6 4 41;496 C6 2 41;480 C5 16 41;496 D#5 16 41;500 D#6 2 41;502 G6 6 41;508 A6 4 41;512 A#6 4 41;512 D#5 16 41;516 A6 4 41;520 F6 4 41;524 D#6 4 41;528 G4 16 41;528 A6 4 41;532 G6 2 41;534 G6 6 41;540 F#6 4 41;544 G4 16 41;544 G6 4 41;548 F#6 4 41;552 E6 2 41;556 D6 4 41;560 G5 4 41;560 G4 16 41;564 B5 2 41;566 D6 6 41;572 F#6 4 41;576 G6 4 41;580 F#6 4 41;584 E6 2 41;588 D6 4 41;576 G4 16 41;592 C5 16 41;592 C6 4 41;604 A6 2 41;606 B6 6 41;608 C5 16 41;612 A6 4 41;616 G6 4 41;620 E6 4 41;624 C6 4 41;624 D#5 16 41;628 D#6 2 41;630 G6 6 41;636 A6 4 41;640 A#6 4 41;644 A6 4 41;648 F6 4 41;652 D#6 4 41;640 D#5 16 41;656 G4 16 41;656 A6 4 41;660 G6 2 41;662 G6 6 41;668 F#6 4 41;672 G6 4 41;672 G4 16 41;676 F#6 2 41;680 E6 4 41;692 B5 2 41;694 D6 6 41;688 G5 4 41;700 F#6 4 41;704 G6 4 41;708 F#6 4 41;712 E6 2 41;716 D6 4 41;688 G4 16 41;704 G4 16 41;724 B6 2 41;726 B6 2 41;728 B6 4 41;732 A6 2 41;734 B6 6 41;740 A6 4 41;744 G6 4 41;720 C5 16 41;736 C5 16 41;748 E6 4 41;752 D#5 16 41;752 C6 4 41;756 D#6 2 41;758 G6 6 41;764 A6 4 41;768 A#6 4 41;772 A6 4 41;768 D#5 16 41;776 F6 4 41;780 D#6 4 41;784 G4 16 41;800 G4 16 41;784 G6 4 41;788 A6 2 41;792 F#6 4 41;796 G6 4 41;800 E6 4 41;804 F#6 2 41;808 D#6 8 41;816 B4 16 41;816 G6 4 41;820 A6 2 41;824 F#6 4 41;828 G6 4 41;832 E6 4 41;836 F#6 2 41;840 D#6 8 41;832 B4 16 41;848 G4 16 41;864 G#4 16 41;848 D6 4 41;852 E6 4 41;856 G6 4 41;860 D7 4 41;864 C7 12 41;876 B6 2 41;878 C7 2 41;880 B6 4 41;884 A6 4 41;888 G6 4 41;892 E6 4 41;896 D#6 4 41;900 A6 8 41;908 B6 4 41;912 G6 32 41;880 A4 16 41;896 C5 16 41;912 G4 32 41;410 D6 2 41;442 G6 2 41;474 G6 2 41;498 C6 2 41;554 E6 2 41;586 E6 2 41;596 E6 4 41;600 G6 2 41;602 G6 2 41;678 F#6 2 41;684 D6 4 41;714 E6 2 41;790 A6 2 41;806 F#6 2 41;822 A6 2 41;838 F#6 2 41;336 B6 4 2;340 B6 4 2;344 B6 4 2;348 B6 4 2;352 B6 4 2;356 B6 4 2;360 B6 4 2;364 B6 4 2;368 B6 4 2;372 B6 4 2;376 B6 4 2;380 B6 4 2;388 E3 3 2;392 E3 3 2;396 E3 3 2;387 E3 1 2;391 E3 1 2;395 E3 1 2;399 E3 1 2;404 B6 4 2;412 B6 4 2;420 B6 4 2;428 B6 4 2;436 B6 4 2;444 B6 4 2;452 B6 4 2;460 B6 4 2;468 B6 4 2;476 B6 4 2;484 B6 4 2;492 B6 4 2;500 B6 4 2;508 B6 4 2;516 B6 4 2;524 B6 4 2;532 B6 4 2;540 B6 4 2;548 B6 4 2;556 B6 4 2;564 B6 4 2;572 B6 4 2;580 B6 4 2;588 B6 4 2;596 B6 4 2;604 B6 4 2;612 B6 4 2;620 B6 4 2;628 B6 4 2;636 B6 4 2;644 B6 4 2;652 B6 4 2;660 B6 4 2;668 B6 4 2;676 B6 4 2;684 B6 4 2;692 B6 4 2;700 B6 4 2;708 B6 4 2;716 B6 4 2;724 B6 4 2;732 B6 4 2;740 B6 4 2;748 B6 4 2;756 B6 4 2;764 B6 4 2;772 B6 4 2;780 B6 4 2;788 B6 4 2;796 B6 4 2;804 B6 4 2;812 B6 4 2;820 B6 4 2;828 B6 4 2;836 B6 4 2;844 B6 4 2;852 B6 4 2;860 B6 4 2;868 B6 4 2;876 B6 4 2;880 B6 1 2;884 B6 4 2;892 B6 4 2;900 B6 4 2;908 B6 4 2;916 B6 4 2;924 B6 4 2;932 B6 4 2;940 B6 4 2;784 B6 1 2;792 B6 1 2;800 B6 1 2;808 B6 1 2;816 B6 1 2;824 B6 1 2;832 B6 1 2;840 B6 1 2;848 B6 1 2;856 B6 1 2;864 B6 1 2;872 B6 1 2;888 B6 1 2;896 B6 1 2;904 B6 1 2;912 B6 1 2;920 B6 1 2'
song=songD

mySong = music(song, pins=[Pin(14)])

while True:
    mySong.tick()
    sleep(0.04)
    
#press interrupt kernel when done

In [ ]:
# The buzzer will stay on as the PWM pin is still turned on. Silence it using :
pwm = PWM(Pin(14))
pwm.duty_u16(0)

In [ ]:
%use micropython
%disconnect